In [20]:
from scipy.stats import norm
from scipy.optimize import brentq
import math
import pandas as pd 
import numpy as np

import requests
from datetime import datetime
from dotenv import load_dotenv
import os
load_dotenv()
REBAR_API_KEY = os.getenv('REBAR_API_KEY')

In [ ]:
#options
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Prices a European option using the Black-Scholes formula.

    Args:
        S (float): Current price of the underlying asset (spot price).
        K (float): Strike price of the option.
        T (float): Time to maturity in years.
        r (float): Risk-free interest rate (annualized).
        sigma (float): Volatility of the underlying asset (annualized).
        option_type (str): Either 'call' or 'put'.

    Returns:
        float: The theoretical option price.
    """
    if T <= 0 or sigma <= 0:
        return max(0.0, S - K) if option_type == 'call' else max(0.0, K - S)

    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def estimate_volatility(price_series):
    """
    Estimates annualized volatility from a series of historical prices.

    Args:
        price_series (List[float]): List of historical prices (chronological).

    Returns:
        float: Annualized volatility.
    """
    log_returns = np.diff(np.log(price_series))
    return np.std(log_returns) * math.sqrt(365)

def time_to_expiry(expiry_timestamp_ms):
    """
    Calculates the time to expiry in years.

    Args:
        expiry_timestamp_ms (int): Expiry timestamp in milliseconds.

    Returns:
        float: Time to expiry in years.
    """
    now = datetime.utcnow().timestamp()
    T = (expiry_timestamp_ms / 1000 - now) / (60 * 60 * 24 * 365)
    return max(T, 0)

In [ ]:
#Rebar data feeds

REBAR_URL = "https://api.rebarlabs.io"
headers = {
        "X-API-Key":REBAR_API_KEY
    }

#inscriptions
def get_inscriptions():
    url =  f"{REBAR_URL}/ordinals/v1/inscriptions"
    res = requests.get(url,headers=headers)
    return res.json()

def get_transfers_for_inscription(inscription_id:str):
    url =  f"{REBAR_URL}/ordinals/v1/inscriptions/{inscription_id}/transfers"
    res = requests.get(url,headers=headers)
    return res.json()

def get_transfer_prices_for_inscription(inscription_id:str):
    transfers= get_transfers_for_inscription(inscription_id)["results"]
    prices = [transfer["value"] for transfer in transfers]
    print(prices)
    return prices

def get_prices_for_brc_20(ticker, limit=20):
    """
    Fetches recent inferred prices from BRC-20 token activity.

    Args:
        ticker (str): BRC-20 token ticker.
        limit (int): Number of recent events to fetch.

    Returns:
        List[float]: Inferred historical prices (chronological order).
    """
    url = f"{REBAR_URL}/ordinals/v1/brc-20/activity"
    params = {"ticker": ticker, "limit": limit}
    response = requests.get(url, headers=headers, params=params).json()

    prices = []
    for tx in response.get("results", []):
        # You must customize this logic depending on how "price" is inferred
        if "price" in tx:
            prices.append(float(tx["price"]))  # Placeholder key
    return prices[::-1]


def price_brc20_option(ticker, strike_price, expiry_timestamp_ms, option_type='call', risk_free_rate=0.0):
    """
    Calculates the theoretical price of a BRC-20 option.

    Args:
        ticker (str): The BRC-20 token ticker.
        strike_price (float): Strike price of the option.
        expiry_timestamp_ms (int): Expiry timestamp in milliseconds.
        option_type (str): 'call' or 'put'.
        risk_free_rate (float): Risk-free rate (default is 0.0).

    Returns:
        float: Option price.
    """
    prices = get_prices_for_brc_20(ticker)
    if len(prices) < 2:
        raise ValueError("Not enough price data to estimate volatility.")

    spot_price = prices[-1]
    sigma = estimate_volatility(prices)
    T = time_to_expiry(expiry_timestamp_ms)

    return black_scholes_price(S=spot_price, K=strike_price, T=T, r=risk_free_rate, sigma=sigma, option_type=option_type)

def price_inscription_option(id, strike_price, expiry_timestamp_ms, option_type='call', risk_free_rate=0.0):
    """
    Calculates the theoretical price of a BRC-20 option.

    Args:
        ticker (str): The BRC-20 token ticker.
        strike_price (float): Strike price of the option.
        expiry_timestamp_ms (int): Expiry timestamp in milliseconds.
        option_type (str): 'call' or 'put'.
        risk_free_rate (float): Risk-free rate (default is 0.0).

    Returns:
        float: Option price.
    """
    prices = get_prices_for_brc_20(id)
    if len(prices) < 2:
        raise ValueError("Not enough price data to estimate volatility.")

    spot_price = prices[-1]
    sigma = estimate_volatility(prices)
    T = time_to_expiry(expiry_timestamp_ms)

    return black_scholes_price(S=spot_price, K=strike_price, T=T, r=risk_free_rate, sigma=sigma, option_type=option_type)


In [ ]:

results = get_inscriptions()["results"]
inscriptions_df = pd.DataFrame(results)
inscriptions_df.head()


5c79e9523e750094c38b4f2072f569ece9caae45cdf6261a00c10b00fd0f0bfdi0


In [18]:
random_ids = inscriptions_df.sample(5, axis=0)['id']
id=random_ids.iloc[0]

transfers= get_transfers_for_inscription(id)["results"]
print(transfers)

[{'block_height': 862701, 'block_hash': '00000000000000000000c5d4ae1736dfe247a96b51979000a894fd48e2bb49e8', 'address': 'bc1pvxcfympr4h6vdy0pfjjtgq4nx0y7jqek366t7j9wdrsuvjx8pwss9vemxt', 'tx_id': '5b26aae57c14b0d740576453535995d556df12e96716c2783948a377ecc2e334', 'location': '5b26aae57c14b0d740576453535995d556df12e96716c2783948a377ecc2e334:0:0', 'output': '5b26aae57c14b0d740576453535995d556df12e96716c2783948a377ecc2e334:0', 'value': '546', 'offset': '0', 'timestamp': 1727202250000}]


In [ ]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the Black-Scholes price of a European call or put option.

    Parameters:
        S (float): Current price of the underlying asset (spot price).
        K (float): Strike price of the option.
        T (float): Time to expiration in years.
        r (float): Annual risk-free interest rate (as a decimal).
        sigma (float): Volatility of the underlying asset (standard deviation of returns).
        option_type (str): Type of the option, either 'call' or 'put'. Defaults to 'call'.

    Returns:
        float: Theoretical price of the option using the Black-Scholes model.

    Notes:
        - If T <= 0 or sigma <= 0, the function returns the intrinsic value of the option.
        - Assumes no dividends and efficient markets.
    """
    if T <= 0 or sigma <= 0:
        return max(0.0, S - K) if option_type == 'call' else max(0.0, K - S)

    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    return S * norm.pdf(d1) * math.sqrt(T)

def implied_volatility(option_price, S, K, T, r, option_type='call'):
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - option_price
    try:
        # Safe bounded root-finding
        return brentq(objective, 1e-6, 3.0, maxiter=1000)
    except (ValueError, RuntimeError):
        return float('nan')